In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastbook import *
from utils import *
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image, display_svg, SVG
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
import seaborn as sn

path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 200
pd.options.display.max_columns = 150

In [3]:
train = pd.read_csv(path/'train.csv', index_col=[0], low_memory=False)

In [4]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,...,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,T,T,T,M0,F,F,F,F,F,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile

In [5]:
train.shape

(590540, 434)

In [24]:
np.linspace(0.1,1,10)

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [33]:
%%time
train_dfs =[]
for frac in np.linspace(0.1,1,10):
    train_dfs.append(train.sample(frac=frac, axis=0, random_state=int(frac*10))
                     .drop(list(train.filter(regex = 'V')), axis = 1))

CPU times: user 15.9 s, sys: 13.8 s, total: 29.7 s
Wall time: 30.7 s


In [34]:
train_dfs[0].shape, train_dfs[1].shape, train_dfs[9].shape

((59054, 95), (118108, 95), (590540, 95))

In [12]:
cat = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38'] 
cont = ['TransactionID', 'TransactionAmt', 'TransactionDT',
        'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 
        'C10', 'C11','C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 
        'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'id_01', 'id_02', 
        'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 
        'id_11']
dep_var = 'isFraud'

In [39]:
len(cont),len(cat)
assert len(cat)+len(cont)==(len(train.drop(list(train.filter(regex = 'V')), axis = 1).columns)-1)

In [13]:
procs = [Categorify, FillMissing]

In [53]:
train_tos = []
for df in train_dfs:
    cut_off = math.floor(df['TransactionDT'].min() + 
                    (df['TransactionDT'].max() - df['TransactionDT'].min())*0.8)
    print(f'{cut_off}')
    cond = df['TransactionDT'] <= cut_off
    train_idx = np.where( cond)[0]
    valid_idx = np.where(~cond)[0]
    splits = (list(train_idx),list(valid_idx))
    to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits,
                   do_setup=True, reduce_memory=True)
    train_tos.append(to)

12665965
12666125
12666184
12666143
12666184
12666184
12666150
12666150
12666184
12666184


In [55]:
lxs       = [] 
ly        = []
lvalid_xs = []
lvalid_y  = []

for to in train_tos:
    xs, y             = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    lxs.append(xs)
    ly.append(y)
    lvalid_xs.append(valid_xs)
    lvalid_y.append(valid_y)

# Function definitions

In [17]:
def rf(**kwargs):
    return RandomForestClassifier(n_jobs=-1, max_samples=2/3, 
                                  oob_score=True, class_weight='balanced', 
                                  max_features='sqrt', **kwargs)

In [18]:
def fit(m, xs=xs, y=y, valid_xs=valid_xs, valid_y=valid_y):
    m.fit(xs, y)
    prob_train = m.predict_proba(xs)[:,1]
    pred_train = m.predict(xs)
    prob_valid = m.predict_proba(valid_xs)[:,1]
    pred_valid = m.predict(valid_xs)
    Train_AROC = roc_auc_score(y, prob_train )
    Valid_AROC = roc_auc_score(valid_y, prob_valid)
    print(f'Train_AROC: {Train_AROC}')
    print(f'Valid_AROC: {Valid_AROC}')
    return prob_train, pred_train, prob_valid, pred_valid

In [88]:
for i in range(10):
    xs       = lxs[i]
    y        = ly[i]
    valid_xs = lvalid_xs[i]
    valid_y  = lvalid_y[i]
    m = rf(n_estimators=200, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
    print(f'{i+1}0 percent of data:')
    _, _, prob_valid, pred_valid = fit(m);

10 percent of data:
Train_AROC: 0.9936468968356427
Valid_AROC: 0.8717041435495829
20 percent of data:
Train_AROC: 0.9936558407732944
Valid_AROC: 0.8718048741604417
30 percent of data:
Train_AROC: 0.9935414209528401
Valid_AROC: 0.8743485268216408
40 percent of data:
Train_AROC: 0.993652959417287
Valid_AROC: 0.8783671049312685
50 percent of data:
Train_AROC: 0.9935574574591254
Valid_AROC: 0.876654957529354
60 percent of data:
Train_AROC: 0.9933612644944267
Valid_AROC: 0.876736579352245
70 percent of data:
Train_AROC: 0.9934432983948709
Valid_AROC: 0.8761605858863585
80 percent of data:
Train_AROC: 0.9934245891375381
Valid_AROC: 0.8685858623359237
90 percent of data:
Train_AROC: 0.9933686830083099
Valid_AROC: 0.8750582818033017
100 percent of data:
Train_AROC: 0.9935746412926895
Valid_AROC: 0.8696999046744464


In [85]:
to = (path/'to_ss.pkl').load()

In [86]:
xs, y             = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [87]:
m = rf(n_estimators=1000, criterion='entropy',
       max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
_, _, prob_valid, pred_valid = fit(m);

Train_AROC: 0.9938572618997983
Valid_AROC: 0.8776082130662617


In [92]:
%%time
train_dfs =[]
for frac in np.linspace(0.01,0.1,10):
    train_dfs.append(train.sample(frac=frac, axis=0, random_state=int(frac*10))
                     .drop(list(train.filter(regex = 'V')), axis = 1))

CPU times: user 6.86 s, sys: 9.04 s, total: 15.9 s
Wall time: 17.5 s


In [93]:
train_tos = []
for df in train_dfs:
    cut_off = math.floor(df['TransactionDT'].min() + 
                    (df['TransactionDT'].max() - df['TransactionDT'].min())*0.8)
    print(f'{cut_off}')
    cond = df['TransactionDT'] <= cut_off
    train_idx = np.where( cond)[0]
    valid_idx = np.where(~cond)[0]
    splits = (list(train_idx),list(valid_idx))
    to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits,
                   do_setup=True, reduce_memory=True)
    train_tos.append(to)

12663858
12663858
12663804
12665238
12665238
12665738
12665738
12665738
12665965
12665965


In [94]:
lxs       = [] 
ly        = []
lvalid_xs = []
lvalid_y  = []

for to in train_tos:
    xs, y             = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    lxs.append(xs)
    ly.append(y)
    lvalid_xs.append(valid_xs)
    lvalid_y.append(valid_y)

In [95]:
for i in range(10):
    xs       = lxs[i]
    y        = ly[i]
    valid_xs = lvalid_xs[i]
    valid_y  = lvalid_y[i]
    m = rf(n_estimators=200, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
    print(f'{i+1}0 percent of data:')
    _, _, prob_valid, pred_valid = fit(m);

10 percent of data:
Train_AROC: 0.9937924379085382
Valid_AROC: 0.8746245123164328
20 percent of data:
Train_AROC: 0.9933130897865212
Valid_AROC: 0.8758387393004436
30 percent of data:
Train_AROC: 0.9935022423338264
Valid_AROC: 0.8701391338150204
40 percent of data:
Train_AROC: 0.9934517338579331
Valid_AROC: 0.8747227860831244
50 percent of data:
Train_AROC: 0.9935402605877512
Valid_AROC: 0.8760287352493805
60 percent of data:
Train_AROC: 0.993792646513498
Valid_AROC: 0.8740578002618449
70 percent of data:
Train_AROC: 0.9933589568020588
Valid_AROC: 0.8756481973861362
80 percent of data:
Train_AROC: 0.9936315252576668
Valid_AROC: 0.8740668086904584
90 percent of data:
Train_AROC: 0.9934460754483984
Valid_AROC: 0.873554147207551
100 percent of data:
Train_AROC: 0.9935653974854081
Valid_AROC: 0.8723374633793728


In [111]:
%%time
train_dfs =[]
for frac in np.linspace(0.001,0.01,10):
    train_dfs.append(train.sample(frac=frac, axis=0, random_state=int(frac*10))
                     .drop(list(train.filter(regex = 'V')), axis = 1))

CPU times: user 4.42 s, sys: 4.04 s, total: 8.46 s
Wall time: 8.46 s


In [112]:
train_tos = []
for df in train_dfs:
    cut_off = math.floor(df['TransactionDT'].min() + 
                    (df['TransactionDT'].max() - df['TransactionDT'].min())*0.8)
    print(f'{cut_off}')
    cond = df['TransactionDT'] <= cut_off
    train_idx = np.where( cond)[0]
    valid_idx = np.where(~cond)[0]
    splits = (list(train_idx),list(valid_idx))
    to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits,
                   do_setup=True, reduce_memory=True)
    train_tos.append(to)

12674342
12674342
12665351
12664585
12663858
12663858
12663858
12663858
12663858
12663858


In [113]:
lxs       = [] 
ly        = []
lvalid_xs = []
lvalid_y  = []

for to in train_tos:
    xs, y             = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    lxs.append(xs)
    ly.append(y)
    lvalid_xs.append(valid_xs)
    lvalid_y.append(valid_y)

In [114]:
for i in range(10):
    xs       = lxs[i]
    y        = ly[i]
    valid_xs = lvalid_xs[i]
    valid_y  = lvalid_y[i]
    m = rf(n_estimators=200, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
    print(f'{i+1}0 percent of data:')
    _, _, prob_valid, pred_valid = fit(m);

10 percent of data:
Train_AROC: 0.994800036303648
Valid_AROC: 0.864560639070443
20 percent of data:
Train_AROC: 0.9947946975318858
Valid_AROC: 0.8632292423142096
30 percent of data:
Train_AROC: 0.994800036303648
Valid_AROC: 0.8704914064391189
40 percent of data:
Train_AROC: 0.9943195468450527
Valid_AROC: 0.8731541999515855
50 percent of data:
Train_AROC: 0.993945832821701
Valid_AROC: 0.8703703703703703
60 percent of data:
Train_AROC: 0.9940205756263714
Valid_AROC: 0.8673444686516583
70 percent of data:
Train_AROC: 0.993913800191128
Valid_AROC: 0.8767852820140402
80 percent of data:
Train_AROC: 0.9945224201720152
Valid_AROC: 0.8821108690389736
90 percent of data:
Train_AROC: 0.9953819624257243
Valid_AROC: 0.8704914064391189
100 percent of data:
Train_AROC: 0.9941593836921877
Valid_AROC: 0.8772694262890341


In [119]:
%%time
train_dfs =[]
for frac in np.linspace(0.001,0.01,10):
    train_dfs.append(train.sample(frac=frac, axis=0, random_state=int(frac*10))
                     .drop(list(train.filter(regex = 'V')), axis = 1))

CPU times: user 4.46 s, sys: 4.14 s, total: 8.6 s
Wall time: 8.7 s


In [120]:
train_tos = []
for df in train_dfs:
    cut_off = math.floor(df['TransactionDT'].min() + 
                    (df['TransactionDT'].max() - df['TransactionDT'].min())*0.8)
    print(f'{cut_off}')
    cond = df['TransactionDT'] <= cut_off
    train_idx = np.where( cond)[0]
    valid_idx = np.where(~cond)[0]
    splits = (list(train_idx),list(valid_idx))
    to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits,
                   do_setup=True, reduce_memory=True)
    train_tos.append(to)

12674342
12674342
12665351
12664585
12663858
12663858
12663858
12663858
12663858
12663858


In [121]:
lxs       = [] 
ly        = []
lvalid_xs = []
lvalid_y  = []

for to in train_tos:
    xs, y             = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    lxs.append(xs)
    ly.append(y)
    lvalid_xs.append(valid_xs)
    lvalid_y.append(valid_y)

In [123]:
for i in range(10):
    xs       = lxs[i]
    y        = ly[i]
    valid_xs = lvalid_xs[i]
    valid_y  = lvalid_y[i]
    m = rf(n_estimators=200, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
    print(f'{i+1}0 percent of data:')
    _, _, prob_valid, pred_valid = fit(m);

10 percent of data:
Train_AROC: 0.9954780603174435
Valid_AROC: 0.8769063180827886
20 percent of data:
Train_AROC: 0.994864101564794
Valid_AROC: 0.8750907770515614
30 percent of data:
Train_AROC: 0.9946398731507831
Valid_AROC: 0.8754538852578068
40 percent of data:
Train_AROC: 0.994800036303648
Valid_AROC: 0.8755749213265553
50 percent of data:
Train_AROC: 0.9952271380446215
Valid_AROC: 0.888888888888889
60 percent of data:
Train_AROC: 0.9939885429957984
Valid_AROC: 0.8764221738077947
70 percent of data:
Train_AROC: 0.9950616361199941
Valid_AROC: 0.8900992495763738
80 percent of data:
Train_AROC: 0.9948213913906968
Valid_AROC: 0.8743645606390704
90 percent of data:
Train_AROC: 0.9951844278705241
Valid_AROC: 0.8764221738077946
100 percent of data:
Train_AROC: 0.9944423385955826
Valid_AROC: 0.8694020818203825


In [84]:
lxs[0]

,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,DeviceType,DeviceInfo,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,dist1_na,dist2_na,D1_na,D2_na,D3_na,D4_na,D5_na,D6_na,D7_na,D8_na,D9_na,D10_na,D11_na,D12_na,D13_na,D14_na,D15_na,id_01_na,id_02_na,id_03_na,id_04_na,id_05_na,id_06_na,id_07_na,id_08_na,id_09_na,id_10_na,id_11_na,TransactionID,TransactionAmt,TransactionDT,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11
40203,2,1926,164,42,4,53,2,122,58,17,17,0,0,0,0,0,0,0,0,0,2,1685,2,35,6,2,2,58,0,100,77,0,0,0,0,0,0,0,2,2,57,97,4,149,3,2,1,1,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,1,1,2,2,2,2,1,3027203,50.000000,998627,8.0,38.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,23.0,10.0,0.0,0.0,42.125000,0.666666,12.0,36.0,0.0,0.0,0.0,47.0,-5.0,157193.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
287920,1,10648,445,68,4,35,3,0,0,55,56,0,0,0,3,0,0,0,0,0,1,1562,2,38,0,1,1,98,0,145,278,0,0,0,0,0,0,0,1,1,0,50,0,0,0,1,1,2,1,2,2,1,2,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,2,1,1,1,3274920,28.337000,7078328,8.0,38.0,3.0,5.0,0.0,1.0,0.0,1.0,1.0,3.0,0.0,5.0,2.0,2.0,5.0,3.0,0.0,97.0,8.0,450.0,29.0,450.0,29.0,104.916664,0.916666,0.0,36.0,450.0,104.0,0.0,254.0,-5.0,105322.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
258681,5,7156,260,42,4,56,3,176,58,17,0,0,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,1,1,1,2,2,2,2,2,1,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,3245681,117.000000,6205699,8.0,38.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,13.0,13.0,13.0,0.0,10.0,0.0,0.0,42.125000,0.666666,13.0,36.0,0.0,0.0,0.0,13.0,-5.0,124688.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
210023,5,11735,483,42,4,102,2,145,58,54,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,2,2,1,2,2,2,2,2,1,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,3197023,2194.949951,4845516,8.0,38.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,97.0,8.0,0.0,10.0,0.0,0.0,42.125000,0.666666,0.0,36.0,0.0,0.0,0.0,0.0,-5.0,124688.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
331849,5,5807,150,42,4,102,3,248,58,54,0,2,2,2,0,0,2,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,1,1,1,1,2,2,2,2,1,1,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,3318849,47.950001,8176611,8.0,38.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,7.0,7.0,7.0,7.0,7.0,0.0,0.0,42.125000,0.666666,7.0,0.0,0.0,0.0,0.0,7.0,-5.0,124688.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336808,3,11398,299,42,1,28,2,122,58,17,17,0,0,0,0,0,0,0,0,0,2,0,2,21,0,2,2,58,0,187,77,0,0,0,0,0,0,0,2,2,0,101,0,0,3,1,1,2,1,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,1,1,2,2,2,2,1,3323808,100.000000,8286432,8.0,38.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,23.0,10.0,0.0,0.0,42.125000,0.666666,12.0,36.0,0.0,0.0,0.0,47.0,-5.0,137391.0,0.0,0.0,1.0,0.0,14.0,-34.0,0.0,0.0,100.0
253181,5,3321,329,42,4,102,3,76,58,17,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,1,1,1,1,2,2,2,2,1,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,3240181,29.950001,6043246,8.0,38.0,4.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,13.0,4.0,219.0,219.0,29.0,306.0,29.0,0.0,0.0,42.125000,0.666666,0.0,36.0,0.0,0.0,0.0,306.0,-5.0,124688.0,0.0,0.0,0.0

In [109]:
%%time
train_dfs =[]
frac = 0.1
for i in range(10):
    train_dfs.append(train.sample(frac=frac, axis=0)
                     .drop(list(train.filter(regex = 'V')), axis = 1))

CPU times: user 6.84 s, sys: 5.37 s, total: 12.2 s
Wall time: 12.3 s


In [102]:
len(train_dfs)

10

In [110]:
train_tos = []
for df in train_dfs:
    cut_off = math.floor(df['TransactionDT'].min() + 
                    (df['TransactionDT'].max() - df['TransactionDT'].min())*0.8)
    print(f'{cut_off}')
    cond = df['TransactionDT'] <= cut_off
    train_idx = np.where( cond)[0]
    valid_idx = np.where(~cond)[0]
    splits = (list(train_idx),list(valid_idx))
    to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits,
                   do_setup=True, reduce_memory=True)
    train_tos.append(to)

12666117
12666126
12665802
12666119
12665946
12666204
12665986
12666259
12665974
12666262


In [111]:
lxs       = [] 
ly        = []
lvalid_xs = []
lvalid_y  = []

for to in train_tos:
    xs, y             = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    lxs.append(xs)
    ly.append(y)
    lvalid_xs.append(valid_xs)
    lvalid_y.append(valid_y)

In [112]:
lxs[0].shape

(48300, 122)

In [113]:
for i in range(10):
    xs       = lxs[i]
    y        = ly[i]
    valid_xs = lvalid_xs[i]
    valid_y  = lvalid_y[i]
    m = rf(n_estimators=200, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
    print(f'{i+1}0 percent of data:')
    _, _, prob_valid, pred_valid = fit(m, xs=xs, y=y, valid_xs=valid_xs, valid_y=valid_y);

10 percent of data:
Train_AROC: 0.9942123887341016
Valid_AROC: 0.8767677056851632
20 percent of data:
Train_AROC: 0.9935122221820062
Valid_AROC: 0.8895629407235576
30 percent of data:
Train_AROC: 0.9935814554452783
Valid_AROC: 0.8821173448548166
40 percent of data:
Train_AROC: 0.9938557215307101
Valid_AROC: 0.8771889520471343
50 percent of data:
Train_AROC: 0.9938209545818791
Valid_AROC: 0.86227837435686
60 percent of data:
Train_AROC: 0.9941783213837703
Valid_AROC: 0.8776670598638175
70 percent of data:
Train_AROC: 0.9941477615649519
Valid_AROC: 0.879592442378079
80 percent of data:
Train_AROC: 0.9944011770098727
Valid_AROC: 0.8805212981286541
90 percent of data:
Train_AROC: 0.9936160039571557
Valid_AROC: 0.8826607725490555
100 percent of data:
Train_AROC: 0.9936833590435952
Valid_AROC: 0.868744127484728


In [81]:
%%time
m = rf(n_estimators=1000, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
_, _, prob_valid, pred_valid = fit(m);

Train_AROC: 0.9952175992348158
Valid_AROC: 0.8493589743589742
CPU times: user 1.94 s, sys: 443 ms, total: 2.38 s
Wall time: 1.82 s


In [85]:
train_ss = pd.read_csv(path/'train_ss.csv', index_col=[0], low_memory=False)

In [86]:
to = (path/'to_ss.pkl').load()

In [87]:
xs, y             = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [88]:
xs.shape, valid_xs.shape, valid_y.sum()

((48491, 122), (10563, 122), 359)

In [97]:
%%time
m = rf(n_estimators=1000, criterion='entropy',
           max_leaf_nodes=750, min_samples_split=30, min_samples_leaf=5)
_, _, prob_valid, pred_valid = fit(m);

Train_AROC: 0.9962936394069823
Valid_AROC: 0.8493589743589745
CPU times: user 2.06 s, sys: 456 ms, total: 2.51 s
Wall time: 1.94 s
